In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<br>
<br>
<br>
<br>

# **Modelo Baseline**

Vamos criar um modelo básico, sem muitas técnicas e métodos, utilizando somente a tabela de **application_train** que já temos disponível, e treinar alguns algoritmos para saber como eles se saem em questão de desempenho, e a partir daí tentar melhorar essa performance, seja criando novas features, seja utilizando outras técnicas, tratamentos e etc.
<br>
<br>
Fazer um modelo baseline é essencial ao treinar um modelo de machine learning, pois fornece uma referência inicial para entender a complexidade do problema, estabelece um benchmark para comparação com modelos mais avançados, identifica problemas nos dados ou formulação do problema e facilita a comunicação com stakeholders ao oferecer uma referência compreensível para discutir desempenho e resultados esperados do modelo.

<br>

## **Pacotes e Bibliotecas**

In [ ]:
# Instalação dos Pacotes.
!pip install colorama > /dev/null
!pip install catboost > /dev/null
!pip install category_encoders > /dev/null

In [ ]:
# Importando as bibliotecas Pandas e Numpy.
import pandas as pd
import numpy as np

# Importando a função train_test_split para a divisão do nosso dataset em treino e teste.
from sklearn.model_selection import train_test_split

# Importando a classe Pipeline para simplificar o processo de pré-processamento.
from sklearn.pipeline import Pipeline

# Importando as bibliotecas necessárias para o tratamento dos dados.
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Importando os algoritmos de Machine Learning, da biblioteca Scikit-Learn, que iremos utilizar nesse projeto.
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Importando as bibliotecas para avaliação dos modelos.
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, classification_report, auc

# Importando a pickle para serialização.
import pickle

# Importando as constantes definidas na biblioteca colorama que permitem alterar as cores e estilos de texto impresso no terminal.
from colorama import Fore, Style

<br>
<br>
<br>
<br>

## **Funções**

In [ ]:
# Função para checar quantidade de linhas e colunas dos conjuntos de treino e teste.
def info_dataframe(treino, teste):
    '''
    Esta função exibe informações sobre a quantidade de linhas e colunas dos conjuntos de treino e teste.

    Parâmetros:
    - treino: DataFrame do conjunto de treino.
    - teste: DataFrame do conjunto de teste.

    Retorna:
    Um print com as informações.
    '''

    # Criando uma borda destacada para o título do conjunto de treino.
    print(Fore.BLUE + '=' * 45)
    print('CONJUNTO DE TREINO'.center(45))
    print('')
    print('Quantidade de linhas:', treino.shape[0])
    print('Quantidade de colunas:', treino.shape[1])
    print('=' * 45 + Style.RESET_ALL)
    print('')

    # Criando uma borda destacada para o título do conjunto de teste.
    print(Fore.GREEN + '=' * 45)
    print('CONJUNTO DE TESTE'.center(45))
    print('')
    print('Quantidade de linhas:', teste.shape[0])
    print('Quantidade de colunas:', teste.shape[1])
    print('=' * 45 + Style.RESET_ALL)



# =================================================================================================================================================== #



# Função para o cálculo das métricas.
def calculate_metrics(nm_modelo, model, X_train, y_train, X_test, y_test):
    '''
    Esta função calcula e imprime várias métricas de desempenho do modelo.

    Parâmetros:
    - nm_modelo: Nome do modelo.
    - model: O modelo treinado.
    - X_train: DataFrame de treino, com as variáveis explicativas.
    - y_train: DataFrame de treino com a coluna de Target.
    - X_test: DataFrame de teste, com as variáveis explicativas.
    - y_test: DataFrame de teste com a coluna de Target.

    Retorna:
    Imprime as métricas de desempenho do modelo.
    '''

    # Faz as predições.
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calcula as métricas para o conjunto de treino.
    accuracy_train = accuracy_score(y_train, y_train_pred)
    precision_train = precision_score(y_train, y_train_pred, zero_division=1)
    recall_train = recall_score(y_train, y_train_pred)
    auc_roc_train = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])

    # Calcula o Índice Gini e Estatística KS para o conjunto de treino.
    probabilities_train = model.predict_proba(X_train)[:, 1]
    df_train = pd.DataFrame({'true_labels': y_train, 'predicted_probs': probabilities_train})
    df_train = df_train.sort_values(by='predicted_probs', ascending=False)
    df_train['cumulative_true'] = df_train['true_labels'].cumsum() / df_train['true_labels'].sum()
    df_train['cumulative_false'] = (1 - df_train['true_labels']).cumsum() / (1 - df_train['true_labels']).sum()
    ks_statistic_train = max(abs(df_train['cumulative_true'] - df_train['cumulative_false']))
    gini_index_train = 2 * auc_roc_train - 1

    # Calcula as métricas para o conjunto de teste.
    accuracy_test = accuracy_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred, zero_division=1)
    recall_test = recall_score(y_test, y_test_pred)
    auc_roc_test = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

    # Calcula o Índice Gini e Estatística KS para o conjunto de teste.
    probabilities_test = model.predict_proba(X_test)[:, 1]
    df_test = pd.DataFrame({'true_labels': y_test, 'predicted_probs': probabilities_test})
    df_test = df_test.sort_values(by='predicted_probs', ascending=False)
    df_test['cumulative_true'] = df_test['true_labels'].cumsum() / df_test['true_labels'].sum()
    df_test['cumulative_false'] = (1 - df_test['true_labels']).cumsum() / (1 - df_test['true_labels']).sum()
    ks_statistic_test = max(abs(df_test['cumulative_true'] - df_test['cumulative_false']))
    gini_index_test = 2 * auc_roc_test - 1


    # Criando o DataFrame.
    metrics_df = pd.DataFrame({
        'Algoritmo': [nm_modelo, nm_modelo],
        'Conjunto': ['Treino', 'Teste'],
        'Acuracia': [accuracy_train, accuracy_test],
        'Precisao': [precision_train, precision_test],
        'Recall': [recall_train, recall_test],
        'AUC_ROC': [auc_roc_train, auc_roc_test],
        'GINI': [gini_index_train, gini_index_test],
        'KS': [ks_statistic_train, ks_statistic_test]
    })

    return metrics_df

<br>
<br>
<br>
<br>

## **Leitura dos Dados**

In [ ]:
# Gravando o diretório em uma variável.
dir = '/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech'

# Lendo nossa tabela de 'application_train'.
df_00 = pd.read_csv(dir + '/database/application_train.csv', sep=',', encoding='latin-1')

df_00.shape

(215257, 172)

In [ ]:
pd.set_option('display.max_columns', None)

df_00.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
0,247330,0,Cash loans,F,N,N,0,157500.0,706410.0,67072.5,679500.0,Unaccompanied,Commercial associate,Higher education,Married,House / apartment,0.032561,-14653,-2062,-8599.0,-2087,NaN,1,1,0,1,1,0,Private service staff,2.0,1,1,WEDNESDAY,13,0,0,0,0,0,0,Services,NaN,0.632424,0.220095,NaN,0.1050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0702,Panel,No,1.0,0.0,1.0,0.0,-1254.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.381340,0.253773,0.205728,0.808261,0.917700,0.487698,0.955921,0.089342,0.519432,0.667806,0.333320,0.873508,0.293837,0.758751,0.972640,0.813237,0.398762,0.060109,0.432021,0.711729,0.455977,0.532977,0.615955,0.005083,0.465449,0.145924,0.026534,0.562217,0.380997,0.634713,0.322195,0.677877,0.518137,0.284267,0.896499,0.260938,0.030923,0.052023,0.969193,0.984378,0.824762,0.333516,0.293260,0.564878,0.115058,0.655605,0.415562,0.092643,0.723331,0.796523
1,425716,1,Cash loans,F,Y,Y,1,121500.0,545040.0,25407.0,450000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.007114,-13995,-2246,-348.0,-172,12.0,1,1,1,1,1,0,Secretaries,3.0,2,2,MONDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.593456,0.695997,0.633032,0.6680,NaN,0.9856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6817,NaN,NaN,0.6807,NaN,0.9856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7102,NaN,NaN,0.6745,NaN,0.9856,NaN

<br>
<br>
<br>
<br>

## **Validação Cruzada tipo Holdout utilizando modo Out-of-Sample**

A validação cruzada é a divisão do dataset em conjuntos de treino e teste e é de extrema importância para avaliarmos, ajustarmos e selecionarmos os modelos de aprendizado de máquina de forma robusta e confiável, garantindo que eles sejam capazes de generalizar para novos dados e resolver efetivamente o problema em questão. Vamos então separar o nosso dataset em 80% para treino e 20% para teste.

In [ ]:
# Separando o Target.
X = df_00.drop(columns=['TARGET'])
y = df_00['TARGET']

In [ ]:
# Dividindo os dados em conjunto de treinamento e teste (80% treino, 20% teste).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

info_dataframe(X_train, X_test)

              CONJUNTO DE TREINO             

Quantidade de linhas: 172205
Quantidade de colunas: 171

              CONJUNTO DE TESTE              

Quantidade de linhas: 43052
Quantidade de colunas: 171


In [ ]:
X_train.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
54931,281575,Cash loans,F,Y,Y,0,202500.0,625500.0,34060.5,625500.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.028663,-19489,-1075,-3962.0,-3011,3.0,1,1,0,1,0,0,Managers,2.0,2,2,THURSDAY,9,0,0,0,0,1,1,Trade: type 7,0.758916,0.526224,0.510090,0.0619,0.0841,0.9771,0.6872,0.0079,0.00,0.1379,0.1667,0.2083,0.0458,0.0471,0.0530,0.0154,0.0456,0.0630,0.0873,0.9772,0.6994,0.0079,0.0000,0.1379,0.1667,0.2083,0.0469,0.0514,0.0552,0.0156,0.0483,0.0625,0.0841,0.9771,0.6914,0.0079,0.00,0.1379,0.1667,0.2083,0.0466,0.0479,0.0539,0.0155,0.0466,reg oper account,block of flats,0.0559,"Stone, brick",No,1.0,0.0,1.0,0.0,-1497.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,0.572098,0.895317,0.666734,0.547289,0.131815,0.762465,0.411189,0.945810,0.907047,0.779933,0.289410,0.226077,0.217647,0.566837,0.395197,0.333989,0.006813,0.461670,0.770898,0.876823,0.744295,0.668739,0.685141,0.659498,0.440569,0.004373,0.791284,0.075672,0.595230,0.235064,0.985042,0.576337,0.350662,0.084907,0.644520,0.487170,0.427249,0.031264,0.699787,0.831006,0.471336,0.264834,0.010173,0.530290,0.102242,0.224055,0.630836,0.543184,0.969463,0.763900
105184,253241,Cash loans,M,N,Y,0,112500.0,278811.0,14364.0,189000.0,"Spouse, partner",Working,Secondary / secondary special,Married,House / apartment,0.030755,-21070,-1099,-5917.0,-4598,NaN,1,1,1,1,0,0,Laborers,2.0,2,2,SUNDAY,9,0,0,0,0,0,0,Business Entity Type 3,NaN,0.698457,0.367291,0.1113,0.0653,0.

In [ ]:
X_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50
196348,243431,Revolving loans,M,N,Y,0,90000.0,180000.0,9000.0,180000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Single / not married,With parents,0.031329,-9579,-489,-9175.0,-1161,NaN,1,1,0,1,0,0,Laborers,1.0,2,2,WEDNESDAY,16,0,0,0,0,0,0,Business Entity Type 3,0.217777,0.634658,0.554947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-393.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,0.553600,0.735979,0.037344,0.338258,0.545242,0.841952,0.024305,0.615976,0.187460,0.588908,0.184133,0.216715,0.295191,0.088721,0.783669,0.926316,0.825076,0.502182,0.088370,0.897170,0.443596,0.505481,0.760015,0.290377,0.252905,0.993413,0.954151,0.111669,0.216014,0.629482,0.292913,0.325685,0.409491,0.000870,0.100324,0.988315,0.332587,0.231480,0.526877,0.678298,0.593565,0.789703,0.254486,0.969768,0.918087,0.939984,0.941608,0.053827,0.571771,0.186664
147976,127962,Cash loans,F,N,N,0,225000.0,781920.0,42547.5,675000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,0.018801,-20151,-3330,-10255.0,-3468,NaN,1,1,0,1,0,0,Laborers,1.0,2,2,MONDAY,11,0,0,0,0,0,0,Business Entity Type 3,0.804014,0.501598,0.384207,0.0082,0.0,0.9687,0.5716,0.0026,0.0,0.0345,0.0417,0.0833,0.0135,0.0067,0.0075,0.0,0.0,0.0084,0.0,0.9687,0.5884,0.

<br>
<br>
<br>
<br>

## **Pipeline de Preparação dos Dados (DataPrep)**

A etapa de preparação dos dados (dataprep) é crucial para o sucesso da modelagem de machine learning, pois os dados precisam estar limpos, organizados e em um formato adequado para serem utilizados pelos algoritmos de aprendizado de máquina. Isso inclui garantir a qualidade dos dados, ajustar o formato para atender aos requisitos dos algoritmos, garantir eficiência computacional, promover interpretabilidade e explicabilidade dos modelos e assegurar generalização e robustez.
<br>
<br>
Vamos utilizar a classe **Pipeline** da biblioteca **Sklearn** para simplificar os processos e treinar os modelos de forma mais rápida, pois nesse momento só queremos visualizar o resultado final, que é o desempenho dos modelos.
<br>
<br>
Vamos identificar as variáveis numéricas e as variáveis categóricas. Para as variáveis categóricas vamos imputar a moda nos valores missing e encodificar os dados com o **Target Encoder**. Para as variáveis numéricas vamos imputar a média nos valores missing e padronizar os dados com **Standard Scaler**.

In [ ]:
# Atribuindo uma variável para as variáveis do tipo 'object' e outra para as variáveis numéricas.
cat_attributes = X_train.select_dtypes(include='object')
num_attributes = X_train.select_dtypes(exclude='object')

# Definindo o pipeline para as variáveis categóricas.
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', TargetEncoder())
])

# Definindo o pipeline para as variáveis numéricas.
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Combinando os pipelines.
preprocessor = ColumnTransformer([
    ('cat', cat_pipe, cat_attributes.columns),
    ('num', num_pipe, num_attributes.columns)
])
processed = Pipeline(steps=[('preprocessor', preprocessor)])

# Aplicando as transformações nos dados de treino e teste.
X_train_processed = pd.DataFrame(processed.fit_transform(X_train, y_train), columns=X_train.columns)
X_test_processed = pd.DataFrame(processed.transform(X_test), columns=X_test.columns)

<br>
<br>
<br>
<br>

## **Treinamento dos Modelos**

Vamos treinar o modelo baseline com os principais algoritmos de machine learning e no final gerar um DataFrame com as principais métricas de cada um deles, para verificarmos qual obteve o melhor desempenho.

In [ ]:
# Criando uma lista com os algoritmos que serão usados para o treinamento do modelo baseline.
algoritmos = [DecisionTreeClassifier(criterion='gini',random_state=42),
              LogisticRegression(solver='liblinear',random_state=42),
              RandomForestClassifier(random_state=42, verbose=False),  # Configurando verbose para False.
              GradientBoostingClassifier(random_state=42),
              XGBClassifier(random_state=42, verbosity=0),  # Configurando verbosity para 0.
              lgb.LGBMClassifier(random_state=42, silent=True, verbose=-1),  # Configurando silent para True e verbose para -1.
              CatBoostClassifier(random_state=42, verbose=False)]  # Configurando verbose para False.

# Iterando sobre os algoritmos.
for algoritmo in algoritmos:
    nome_algoritmo = algoritmo.__class__.__name__

    # Treino do modelo.
    algoritmo.fit(X_train_processed, y_train)

    # Avaliar modelo.
    metricas = calculate_metrics(nome_algoritmo, algoritmo, X_train_processed, y_train, X_test_processed, y_test)
    display(metricas)

,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,DecisionTreeClassifier,Treino,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,DecisionTreeClassifier,Teste,0.851482,0.141109,0.160511,0.536759,0.073519,0.073676


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,LogisticRegression,Treino,0.919515,0.557971,0.011086,0.747434,0.494868,0.369245
1,LogisticRegression,Teste,0.918215,0.493151,0.010227,0.742754,0.485508,0.358696


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,RandomForestClassifier,Treino,0.999959,1.0,0.999496,1.000000,1.000000,1.000000
1,RandomForestClassifier,Teste,0.918238,1.0,0.000000,0.693526,0.387051,0.290994


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,GradientBoostingClassifier,Treino,0.920107,0.677249,0.018428,0.764702,0.529403,0.394877
1,GradientBoostingClassifier,Teste,0.918540,0.580247,0.013352,0.749314,0.498627,0.375040


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,XGBClassifier,Treino,0.930048,0.932521,0.143248,0.917727,0.835454,0.666612
1,XGBClassifier,Teste,0.917704,0.461017,0.038636,0.728227,0.456454,0.344353


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,LGBMClassifier,Treino,0.921152,0.825726,0.028650,0.826700,0.653399,0.496511
1,LGBMClassifier,Teste,0.918494,0.556701,0.015341,0.748999,0.497998,0.369684


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,CatBoostClassifier,Treino,0.928411,0.989362,0.113807,0.873756,0.747511,0.575303
1,CatBoostClassifier,Teste,0.918355,0.515337,0.023864,0.748419,0.496838,0.374209


É importante observar todas as métricas e como elas se comportam para cada modelo treinado, porém nesse caso, como estamos tentando resolver um problema de concessão de crédito e risco de inadimplência, iremos focar na AUC_ROC, Gini e KS, porque além de serem métricas pedidas para serem priorizadas pelo time de negócios, são métricas que possuem uma boa capacidade discriminativa do modelo e sua capacidade de distinguir entre bons e maus pagadores, ajudando assim na tomada de decisão precisa e eficaz.
<br>
<br>
Então colocando um foco maior nessas métricas, podemos chegar a conclusão que o modelo treinado com o algoritmo **GradientBoostingClassifier** obteve melhor desempenho nos dados de teste, ante os demais modelos, além de ter métricas consistentes entre os conjuntos de treino e teste, que mostra que o modelo é robusto e confiável, capaz de generalizar bem para novos dados e fazer previsões precisas em situações do mundo real.
<br>
<br>
O modelo escolhido para ser o nosso modelo baseline (modelo de entrada), será o modelo treinado com o algoritmo **GradientBoostingClassifier** e suas métricas servirão de parâmetro quando formos aplicar técnicas e métodos afim de melhorar essas métricas.

In [ ]:
algoritmos = [GradientBoostingClassifier(random_state=42)]

# Iterando sobre os algoritmos.
for algoritmo in algoritmos:
    nome_algoritmo = algoritmo.__class__.__name__

    # Treino do modelo.
    algoritmo.fit(X_train_processed, y_train)

    # Avaliar modelo.
    metricas_modelo_baseline = calculate_metrics(nome_algoritmo, algoritmo, X_train_processed, y_train, X_test_processed, y_test)
    display(metricas_modelo_baseline)

,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,GradientBoostingClassifier,Treino,0.920107,0.677249,0.018428,0.764702,0.529403,0.394877
1,GradientBoostingClassifier,Teste,0.918540,0.580247,0.013352,0.749314,0.498627,0.375040


<br>
<br>
<br>
<br>

## **Salvamento do Modelo escolhido em um Arquivo pkl**

Vamos salvar o modelo treinado com o algoritmo **GradientBoostingClassifier** em um arquivo serealizado do tipo pickle, para que ele possa ser usado depois caso seja necessário.

In [ ]:
with open(dir + '/artefatos/baseline/baseline_gradient_boosting_v1.pkl', 'wb') as file:
  pickle.dump(algoritmo, file)